# FFmpeg demo: video encoding

In [4]:
# Download a static FFmpeg build and add it to PATH.
%run 'util/load-ffmpeg.ipynb'
print('Done!')

/usr/local/bin/ffmpeg
Done!


In [5]:
# [optional] Print input video information. You can upload another video file and change the file name below.
!ffprobe -hide_banner input.mp4
print('Done!')

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'input.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
  Duration: 00:00:10.02, start: 0.000000, bitrate: 2018 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 864x360 [SAR 1:1 DAR 12:5], 1885 kb/s, 24 fps, 24 tbr, 12288 tbn, 48 tbc (default)
    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 128 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
Done!


In [6]:
# Compress input.mp4 to a target bitrate and save as output.mp4. 
# Change the command line flags as you need for different encoding settings.

!ffmpeg -hide_banner -i input.mp4 -b:v 200k -y output.mp4
print('Done!')

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'input.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
  Duration: 00:00:10.02, start: 0.000000, bitrate: 2018 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 864x360 [SAR 1:1 DAR 12:5], 1885 kb/s, 24 fps, 24 tbr, 12288 tbn, 48 tbc (default)
    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 128 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
Stream mapping:
  Stream #0:0 -> #0:0 (h264 (native) -> h264 (libx264))
  Stream #0:1 -> #0:1 (aac (native) -> aac (native))
Press [q] to stop, [?] for help
[libx264 @ 0x7f8de1815000] using SAR=1/1
[libx264 @ 0x7f8de1815000] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x7f8de1815000] profile High, level 3.0
[libx264 @ 0x7f8de1815000] 264 - core 155

In [7]:
# Display both the source (top) video and encoded (bottom) video.
# Sometimes the two players get out of sync. Press "sync" to mitigate that.

%run util/play-video.ipynb --ref input.mp4 --tar output.mp4
HTML(html)

In [11]:
# Calculate PSNR between input.mp4 and output.mp4
!ffmpeg -hide_banner -i output.mp4 -i input.mp4 -lavfi psnr -f null - 2>&1 | grep "Parsed_psnr_0"

# Calculate SSIM between input.mp4 and output.mp4
!ffmpeg -hide_banner -i output.mp4 -i input.mp4 -lavfi ssim -f null - 2>&1 | grep "Parsed_ssim_0"

# Calculate VMAF between input.mp4 (ref) and output.mp4 (target).
!ffmpeg -hide_banner -i output.mp4 -i input.mp4 \
  -lavfi "libvmaf=model_path={ffmdir[0]}/model/vmaf_v0.6.1.pkl" -f null - 2>&1 | grep 'VMAF' \
  || echo 'VMAF calculation failed!'

[Parsed_psnr_0 @ 0x7fd099600340] PSNR y:33.079084 u:40.324532 v:42.078959 average:34.511333 min:32.170013 max:38.685101
[Parsed_ssim_0 @ 0x7fc62fd37680] SSIM Y:0.921991 (11.078568) U:0.939237 (12.163643) V:0.958136 (13.781555) All:0.930890 (11.604575)
VMAF calculation failed!
